In [ ]:
from SL_C0de.SOLVER import Precomputation

from scipy import io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Loading Ice data

data=io.loadmat(r'I:/SL_C0de/SL_C0de_data/ice6G122k') #load the file, can be modified to load an other file.
ice_in=data['ice6g']# ice_in[temps, latitude, longitude] hauteur de la glace
ice_time=data['ice_time']
ice_time=ice_time.squeeze()
ice_time=np.append(ice_time[0]+1,ice_time)
ice_lon =data['ice_long'].squeeze()
ice_lat =data['ice_lat'].squeeze()[::-1]
# The ice is not in the good shape. We have to derivate it : 
ice_0=ice_in.T[0,:,:].copy()
ice_in=np.diff(ice_in.T,axis=0)
ice_in=np.concatenate((np.expand_dims(ice_0,axis=0),ice_in),axis=0)

ice_grid=dict(name='ICE_ICE6G', grid=ice_in, time=ice_time,lon=ice_lon,lat=ice_lat)

# Loading sediment loading data

sed_ncgrid = pd.read_csv(r'I:/SL_C0de/SL_C0de_data/AYS2_sed.csv') 

lon=np.array(sed_ncgrid["POINT_X.N.19.11"])
lat=np.array(sed_ncgrid["POINT_Y.N.19.11"])
sed=np.array(sed_ncgrid[:]).T/4
sed=sed[3:,:]
#sed=sed[::-1,:]*0 # add a zero sediment model
sed_time_step=np.arange(26.25,-0.25,-0.25)

area=(93,98,13,20)

sed_grid=dict(name='AYS2', grid=sed, time=sed_time_step,lon=lon,lat=lat,frame=area)

# Loading topographic data

data=io.loadmat(r'I:/SL_C0de/SL_C0de_data/topo_SL') #load the file, can be modified to load an other file.
topo_grid=data['topo_bed'][::2,1::2].squeeze()
topo_lat=data['lat_topo'].squeeze()[::-2]
topo_lon=data['lon_topo'].squeeze()[1::2]

topo_grid=dict(name='topo_SL',grid=topo_grid,lon=topo_lon,lat=topo_lat)




In [ ]:
# Precompute the full data to prepare the next resolution of the SLE

stop=26 # define the number of time steps
step=0.25
time_step=np.arange(start=stop,stop=-step,step=-step) # in kyr to present
time_step[-1]=0
maxdeg=384 # define the maximum degree for spherical harmonics.
Output_way='I:/SL_C0de/Interpolated_grid/Test_0.25kyr'
plot=True

Precomputation(ice_grid,sed_grid,topo_grid,Output_way,stop=stop,step=step,maxdeg=maxdeg,plot=plot)


In [ ]:
# Solve the SLE in forward modeling way 
from SL_C0de.SOLVER import SLE_forward_modeling

Input_way='H:/SL_C0de/Interpolated_grid/Test'
ice_name='ICE_ICE6G'
sediment_name='AYS2'
topo_name='topo_SL'
ocean_name='OCE'
love_way='H:/SL_C0de/SL_C0de_data/variable_lm_32'
love_file='VM5a.l32.um21.lm22.449'
conv_lim=10e-10
Output_way='H:/SL_C0de/Interpolated_grid/Test'

SLE_forward_modeling(Input_way,ice_name,sediment_name,topo_name,ocean_name,love_way,love_file,conv_lim,Output_way)

In [ ]:
from SL_C0de.SOLVER import SLE_solver

Input_way='I:/SL_C0de/Interpolated_grid/Test_6kyr'
ice_name='ICE_ICE6G'
sediment_name='AYS2'
topo_name='topo_SL'
ocean_name='OCE'
love_way='I:/SL_C0de/SL_C0de_data/'
love_file='VM5a_122'
conv_lim=10e-10
topo_lim=1
Output_way='I:/SL_C0de/Interpolated_grid/Test_6kyr'

load=SLE_solver(Input_way,ice_name,sediment_name,topo_name,ocean_name,love_way,love_file,topo_lim,conv_lim,Output_way)

In [ ]:
from SL_C0de.SOLVER import Post_process

Input_way='I:/SL_C0de/Interpolated_grid/Test_1024/model_output'
sed_name='AYS2'
ice_name='ICE_ICE6G'
ocean_name='OCE'
topo_name='topo_SL'
love_way='I:/SL_C0de/SL_C0de_data'
Post_process(Input_way,sed_name,ice_name,ocean_name,topo_name,love_way)

In [ ]:
import numpy as np
Input_way='H:/SL_C0de/Interpolated_grid/Test_512/model_output/VM5A.l32.um20.25.lm22.699'
plot=dict(plot=True,
          times=[0,1,15,20],
          frames=[(-179,179,-89,89),(93,98,13,20)],
          frames_resolution=[1024,2048],
          #frames_min_max=np.array([[-30,200,-3,20],[-2,3,-0.2,0.1]]),
          frames_min_max=np.array([[None,None,None,None],[None,None,None,None]]),
          contours_v=[[[i for i in range(-30,-10,10)]+[i for i in range(-10,10,1)]+[i for i in range(10,30+1,10)]+[100,200],
                       [i/10 for i in range(-18,0,1)]+[i/20 for i in range(0,10,1)]],
                       [[-3,-0.1,0,0.1,3,10,20],[-0.6,-0.3,-0.2,-0.15,-0.1,0]]],
          #contours_v=[[[None],[None]],[[None],[None]]],
          transects=[(21.63,96.01,11.1235,96.04),(15.347829, 89.811065,15.531321, 99.098801)],
          point_density=[512,512],
          transect_min_max=[(-2,0.5,-0.7,1),(-3,2,-0.7,1)],
          points=np.array([[16.8,96],[15,96.25],[15.9609036,95.4758372],[16.0318760,94.8894641],[16.3469326,95.2544328]]),
        )

In [ ]:
from SL_C0de.SOLVER import plot_model_result_map
plot_model_result_map(Input_way,plot)

In [ ]:
from SL_C0de.SOLVER import plot_model_result_cross_section
plot_model_result_cross_section(Input_way,plot)

In [ ]:
from SL_C0de.SOLVER import plot_model_output_points
plot_model_output_points(Input_way,plot)

In [ ]:
from SL_C0de.SOLVER import plot_model_RSL
plot_model_RSL(plot,Input_way,Input_way)

In [ ]:
from SL_C0de.SOLVER import export_to_netcdf 
export_to_netcdf(Input_way,0,plot['frames_resolution'][1],plot['frames'][1],Input_way+'\LOAD','SEDIMENT','LOAD')


In [ ]:
from SL_C0de.grid import LOAD_TIME_GRID
Input_way="H:\SL_C0de\Interpolated_grid\Test_512\model_output\VM5a.l32.um20.25.lm22.699\LOAD\TOTAL_GEOID"
geoid_grid=LOAD_TIME_GRID(from_file=(True,Input_way))

In [ ]:
import numpy as np
geoid_grid.height_time_coeff=geoid_grid.elastic_deformation+geoid_grid.viscuous_deformation
geoid_grid.point_time(np.array([[15,96.25]]))

In [ ]:
from SL_C0de.SOLVER import calculate_dESL
Input_way="H:\SL_C0de\Interpolated_grid\Test_512\model_output\VM5a.l32.um20.25.lm22.699"
calculate_dESL(Input_way)